
Imports:

In [6]:
try:
    import torchtt as tntt
except:
    pass
import torch as tn
import datetime

Set the size of the problem:

In [7]:
dtype = tn.float64 
n =  262 

In [8]:
A1d = -2*tn.eye(n, dtype = dtype)+tn.diag(tn.ones(n-1,dtype = dtype),-1)+tn.diag(tn.ones(n-1,dtype = dtype),1)
A1d[0,1] = 0
A1d[-1,-2] = 0
A1d /= (n-1)
A1d = tntt.TT(A1d, [(n,n)])
# construct 4d TT-matrix
A = A1d**tntt.eye([n,n,n]) +  tntt.eye([n])**A1d**tntt.eye([n,n]) + tntt.eye([n,n])**A1d**tntt.eye([n]) + tntt.eye([n,n,n])**A1d

In [9]:
b1d = tn.ones(n, dtype=dtype)
b1d[0] = 0
b1d[-1] = 0
b1d = tntt.TT(b1d)
b = b1d**b1d**b1d**b1d

In [10]:
time = datetime.datetime.now()
x = tntt.solvers.amen_solve(A, b, x0 = b,nswp = 20,eps = 1e-8,verbose=1, max_full=100, preconditioner=None)
time = datetime.datetime.now() - time
print('Relative residual: ',(A@x-b).norm()/b.norm())
print('Solver time: ',time)

NameError: name 'TT' is not defined

In [ ]:
if tn.cuda.is_available():
    time = datetime.datetime.now()
    x = tntt.solvers.amen_solve(A.cuda(),b.cuda(),x0 = b.cuda(),nswp = 20,eps = 1e-8,verbose=1, preconditioner=None)
    time = datetime.datetime.now() - time
    x = x.cpu()
    print('Relative residual: ',(A@x-b).norm()/b.norm())
    print('Solver time: ',time)
else:
    print('GPU not available...')